## <span style="color:grey">Initialization</span>

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime
import logging
import copy 
import os
from multiprocessing import Process
warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

today = datetime.datetime.now()
#configuration for downloading data
days =  1
years = 5

if today.weekday() >= 5:
    today = today - datetime.timedelta(days = today.weekday() - 4)
    
#today = datetime.datetime(2019, 4, 8, 15, 45, 0)


toDate = today
#toDate = today - datetime.timedelta(days=3)
fromDate = toDate - datetime.timedelta(days = days)


#if today.weekday() == 0:
#    fromDate = toDate - datetime.timedelta(days = 3)

freq = '1min'

mode = "Live" # Backtesting/Live/Algo
offline = False
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

### Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

logging.basicConfig(filename="log/live_log.log", filemode="a", level=logging.DEBUG, format="[%(asctime)s: %(levelname)s]:%(message)s")
logger=logging.getLogger()

kite = KiteConnect(api_key=KiteAPIKey)
reauthentication = False

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])
kite.set_access_token(access_token[0])

try:
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.get(kite.login_url())
    reauthentication = True
    

In [3]:
if reauthentication:
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()
    driver.close()

### Download NSE Stock Master List

In [4]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

In [5]:
instruments_df[instruments_df.name.str.contains('.*NIFTY 50.*')]

,instrument_token,exchange_token,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
tradingsymbol,,,,,,,,,,,
NIFTY 50,256265,1001,NIFTY 50,0.0,,0.0,0.00,0,EQ,NSE-INDICES,NSE
NIFTY 50,256265,1001,NIFTY 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
SETFNIF50,2605057,10176,SBI-ETF NIFTY 50,0.0,,0.0,0.01,1,EQ,NSE,NSE
NIFTYEES,2319105,9059,EDELWEISS ETF - NIFTY 50,0.0,,0.0,0.01,1,EQ,NSE,NSE


### Filter Stocks - portfolio maker

In [6]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

In [7]:
portfolio = pd.DataFrame(downloadlist.tolist())

In [8]:
portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC','NDTV'])

#portfolio = portfolio.append(pd.DataFrame(data=['NIFTY 50']))

portfolio = pd.DataFrame(['TATAMOTORS','WIPRO'])

#portfolio = pd.DataFrame(downloadlist)
#portfolio = pd.DataFrame(['BAJAJ-AUTO','CIPLA'])

#portfolio = pd.DataFrame(['BAJAJ-AUTO'])
portfolioToken = portfolio[0].apply(EQTOKEN)

In [9]:
portfolio

,0
0,TATAMOTORS
1,WIPRO


### Download Historical Data - Equity

In [10]:
def portfolioDownload(stocklist, toDate):
    stocklist_df = pd.DataFrame()
    for index, row in stocklist.iterrows():
        symbol = row[0]
        logging.info("Downloading data for: "+symbol)
        temp_data = downloadData(symbol,  toDate - dt.timedelta(days = 5), toDate)
        temp_data['symbol'] = symbol
        temp_data.set_index(['symbol',temp_data.index], inplace=True)
        #print(temp_data)
        stocklist_df = stocklist_df.append(temp_data)
    
    #print(stocklist_df)
    return stocklist_df    

if False:
    portfolio_df = pd.DataFrame()
    portfolio_df = portfolioDownload(portfolio, today)
    portfolio_df.to_hdf('kite_data/kite_data.h5', key="portfolio",mode='a')

#portfolio_df.loc['HDFC']

### Batch Downloader

In [11]:
def batchDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)

        raw_data = downloadData(symbol, fromDate, toDate)
        raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")

        raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
        raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

### Incremental Download - untested

In [12]:
def incrementalDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol,mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate)
        tempData = tempData.append(raw_data)
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')
        
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate, freq="day")
        
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    
    

### Historical Data storage strategy

* minute level data and day level ohlc data is stored in the hd5
* symbol name is used as a key in the hd5 file system

### Tick Data storage strategy
- Tick data should not be merged with the downloaded historical data
- At the start of the trading session, last 60 candles from the historical data is fetched and stored in the dataframe for holding live data
- Two DataFrames ares created for storing live data: 1 for storing timestamp and LTP, another for OHLC
- OHLC data is created by grouping data first based on stock symbol and then based on timestamp(seconds and miliseconds are ignored)
- Streaming data from all the stocks in portfolio are strored in a single table. During post-procession subset of the master table sliced based on symbol is used
- Streaming data is resampled for minute frequency every minute to convert it to OHLC data which is stored in another dataframe
- Symbol is used as a key for the streaming OHLC dataframe storage

### Load Data from disk

##  <span style="color:green">Trading Strategies </span>

In [13]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0

# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a, days: ROC(a,days) >= 0.01
#TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        iRSI = RSI(CLOSE, timeperiod=14)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
        #atr = ATR(CLOSE)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def long_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACD, iMACDsignal), columns=["buy"])
    
    
    def short_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACDsignal, iMACD), columns=["sell"])
    
    def long_pa(self):
        return pd.DataFrame(TREND_DOWN(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
                            #| (CDLABANDONEDBABY(OPEN, HIGH, LOW, CLOSE) != 0) 
                            ), columns=["buy"])
    def short_pa(self):
        return pd.DataFrame( TREND_UP(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
                            #| (CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0)
                            ) , columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_pa() # |self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_pa() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    def tradeRecommendation(self):
        buy, sell = self.tradeDecision()
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        logging.debug(str(timeNow))
        logging.debug(buy.tail(3))
        logging.debug(sell.tail(3))
        if (delT1 == buy.index[-1]) | (delT2 == buy.index[-1]) :
            return "buy,"+str(timeNow) +","+str(buy.iloc[-1]['low'])+","+str(buy.iloc[-1]['close'])+"\n"
        elif (delT1 == sell.index[-1]) | (delT2 == sell.index[-1]) :  
            return "sell,"+str(timeNow) +","+str(sell.iloc[-1]['high'])+","+str(sell.iloc[-1]['close'])+"\n"
        return ""

    
    def tradeBot(self, symbol):
        global STATE
        buy, sell = self.tradeDecision()
        logger.debug(symbol+","+STATE)
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        
        price = CLOSE[-1]
        trigger = CLOSE[-1]
        #stoploss = SD[-1]
        stoploss = 0
        squareoff = 3 * SD[-1] 
        
        logger.debug(buy)
        logger.debug(sell)
        
        if buy.empty | sell.empty:
            return
        
        if (buy.empty != True) & ((delT1 == buy.index[-1]) | (delT2 == buy.index[-1])) & (STATE != "SELL"):
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_sl(symbol, price, trigger, quantity=1)
            STATE = "SELL"
        elif (sell.empty != True) & ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) & (STATE != "BUY"):
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_sl(symbol, price, trigger, quantity=1)
            
    def papertradeBot(self, symbol, timestamp):
        global STATE, profit, squareoff, stoploss
        buy, sell = self.tradeDecision()
        #logger.debug(buy)

        timeNow = timestamp
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)

        #logger.debug(timeNow)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = 2*SD[-1]
        squareoff = 3 * SD[-1] 
        
        if LOW[-1]<=stoploss:
            profit = profit - stoploss
            STATE = "BUY"
            return
        
        #if HIGH[-1] >= squareoff:
        #    profit = profit - squareoff
        #    STATE = "BUY"
        #    return

        if buy.empty | sell.empty:
            return
        
        if ((delT1 == buy.index[-1]) or (delT2 == buy.index[-1])) and (STATE != "SELL"):
            profit = profit - price
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, 0, quantity=1)
            STATE = "SELL"
        elif ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) and (STATE != "BUY"):
            profit = profit + price
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, 0, quantity=1)
            
        
papertrade = False
if (mode=="Algo") & (papertrade == False):
    global fig
    noofcandles = 60
    x = "HDFC"
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    #logger.debug(temp_data)
    #temp_data = downloadData(x, fromDate, toDate, "day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    #temp_data = aroon(temp_data,5, True)
    temp_data = stoch(temp_data,5, True)
    temp_data = sma(temp_data,1,True)
    obv(temp_data,2, True)
    

    #print(temp_data.tail(200))
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    #print(buy)
    #print(sell)
    #print(CLOSE)
    
    #a.tradeRecommendation()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)


In [14]:
papertrade = True
if papertrade & (mode != "Algo"):
    profit = 0
    STATE = "BUY"
    squareoff = 0
    stoploss = 0
    global fig
    noofcandles = 60
    x = "WIPRO"
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    rowSize = temp_data.shape[0]
    
    for i in np.linspace(120,rowSize-1, rowSize-121, dtype="int64"):
        a = algoTrade(temp_data.iloc[(i-120):i])
        a.papertradeBot(x, temp_data.index[i])
        #print(temp_data.iloc[(i-30):i])
    
    print(profit)

-286.8


In [30]:
STATE = "BUY"

Unhandled Error
Traceback (most recent call last):
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\log.py", line 103, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\log.py", line 86, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\context.py", line 122, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\context.py", line 85, in callWithContext
    return func(*args,**kw)
--- <exception caught here> ---
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\internet\selectreactor.py", line 149, in _doReadOrWrite
    why = getattr(selectable, me

Unhandled Error
Traceback (most recent call last):
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\log.py", line 103, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\log.py", line 86, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\context.py", line 122, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\python\context.py", line 85, in callWithContext
    return func(*args,**kw)
--- <exception caught here> ---
  File "C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\twisted\internet\selectreactor.py", line 149, in _doReadOrWrite
    why = getattr(selectable, me

In [21]:
temp_data = downloadData('HDFC', fromDate = toDate - dt.timedelta(days=365), toDate= toDate, freq="day")

In [16]:
temp_data['atr'] = ATR(temp_data.high, temp_data.low, temp_data.close)
#temp_data['sma'] = SMA(temp_data.close)
temp_data['sd-'] = temp_data.shift(1).close - STDDEV(temp_data.shift(1).close)
temp_data['sd+'] = temp_data.shift(1).close + STDDEV(temp_data.shift(1).close)
temp_data['2sd-'] = temp_data.shift(1).close - 2*STDDEV(temp_data.shift(1).close)
temp_data['2sd+'] = temp_data.shift(1).close + 2* STDDEV(temp_data.shift(1).close)

from talib import TRANGE, NATR

temp_data['tr'] = TRANGE(temp_data.high, temp_data.low, temp_data.close)
temp_data['natr'] = NATR(temp_data.high, temp_data.low, temp_data.close)

## <span style="color:blue">BackTesting</span>

In [17]:
logger.setLevel(logging.DEBUG)

### Multithreaded

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    fig = ""
    temp_data = downloadData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #print(temp_data.head())
    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    obv(temp_data,2, True)

    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData(x, True)

if __name__ == '__main__':
    #if True | (mode == "Algo"):
    logger.debug("Hello")
    p = Pool(10)
    p.map_async(threadedBatchAnalysis, portfolio[0])
    #p.apply_async(threadedBatchAnalysis, ['SBIN'])
    #portfolio
    
#threadedBatchAnalysis()

In [33]:
import gc
gc.get_count()

(64, 1, 6)

In [34]:
gc.collect()

15062

### Backtesting - Single Threaded

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
def batchAnalysis(portfolio=portfolio):
    global fig, noofcandles
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row[0] #row.iloc[:,0]
        temp_data = downloadData(symbol, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
        
        #temp_data['symbol'] = symbol
        #temp_data.set_index(['temp_data.index], inplace=True)
        #logging.debug(temp_data)
        
        yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = temp_data.index[-1*noofcandles]
        xMax = temp_data.index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

batchAnalysis()

NetworkException: Request preparation failed (kitetrade-common).

## <span style="color:red"> Live Trading</span>

### Order Management

In [15]:
papertrade = False

In [16]:
papertrade =True

#logging.critical("BUY"+symbol)
def buy_slm(symbol, price, trigger,quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SLM,
                                product=kite.PRODUCT_MIS,
                                trigger_price=round(trigger,1),
                                #stoploss=round(stoploss,1),
                                #price=price,
                                variety=kite.VARIETY_REGULAR
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))


def sell_slm(symbol, price, trigger, quantity=1):
    
    logging.critical("SELL: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
       
    if papertrade:
         return
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_SLM,
                            product=kite.PRODUCT_MIS,
                            trigger_price=round(trigger,1),
                            #price=price,
                            variety=kite.VARIETY_REGULAR)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))

def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("SELL: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def getOrders():    
    # Fetch all orders
    return pd.DataFrame(kite.orders())

def cancelOrder(orderId):
    if papertrade:
        logging.critical("In Paper Trade Mode: Order cancellation not possible")
        return
    
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=orderId, parent_order_id=None)    
    except Exception as e:
        logging.info("Order Cancellation failed: {}".format(e.message))

In [15]:
profit

NameError: name 'profit' is not defined

In [22]:
round(0.8133879766952922,2) - round(0.8133879766952922,2)

0.0

In [41]:
buy_sl("CIPLA",565.5, 1)

In [39]:
sell_sl("CIPLA",569,0)

In [34]:
cancelOrder(190412002140702)

In [41]:
if (mode == "Live"):
    positions_df = pd.DataFrame(kite.positions()['net'])
    positions_df

In [37]:
if (mode == "Live"):
    orders_df = getOrders()
    if orders_df.empty != True:
        orders_df[orders_df['status']!=''][['order_id','exchange','tradingsymbol','transaction_type','price','trigger_price', 'quantity', 'order_type', 'product', 'status']]
    



In [44]:
logger.debug(positions_df)

In [ ]:
if mode == "Live" & False:
    kite.modify_order(variety=kite.VARIETY_REGULAR, 
                  order_id=190411002378318, quantity=1, price=282, 
                  order_type=kite.ORDER_TYPE_SL)



In [24]:
logger.setLevel(logging.CRITICAL)

### WebSocket: Tick Handler and sampler

In [17]:
import datetime

def resample(ws, freq="1min"):
    #F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
      
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()

        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled)
        
    #logger.info(index+":\n"+str(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(240)))

    ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    a = algoTrade(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(120))
    #reco_df = a.tradeRecommendation()
    a.tradeBot(portfolio.iloc[-1,0])
    
    #try:
    #    if reco_df != "":
    #        F.write(reco_df)
    #except   Exception as e:
    #    logging.critical("Not able to resample and analyse data"+str(e))
    #finally:
    #    F.close()
        



def ticksHandler(ws, ticks):
    timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    #logging.debug("Prev: "+str(ws.prevtimeStamp)+"\nNow: "+str(timeStamp))
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
        
        timeStamp = tick_df.index[0][-1].to_pydatetime()
        
        #logging.info(str(timeStamp - ws.prevtimeStamp))
        #print(type(timeStamp))
        
        #logging.debug("Exception: ticksHandler: "+str(tick_df))
        #timeStamp = tick_df.loc[portfolio.iloc[-1,0]].index[-1]
    except  Exception as e:
        logging.debug("Exception: ticksHandler: "+str(e)+str(tick_df))
        
    if( (timeStamp - ws.prevtimeStamp) >= datetime.timedelta(minutes=1)):
        ws.prevtimeStamp = timeStamp
        
        #ws.cbufferSampling = ws.cbufferAccum.copy(deep=True) #Copy accumulated stream to sampling buffer
        #del ws.cbufferAccum
        #gc.collect()
        #ws.cbufferAccum = pd.DataFrame() #Reset accumulation buffer
        #logging.debug(ws.cbufferSampling)
        resample(ws)
    
    #ws.cbufferAccum = ws.cbufferAccum.append(tick_df)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)
    #ws.LiveStream.to_csv("kite_data/livestream.csv", mode='a')

def orderNotification(ws,data):
    logger.debug(data)

def initTrade(ws):
    ws.prevtimeStamp = datetime.datetime.now() - dt.timedelta(minutes=10)
    toDate = datetime.datetime.now()
    #ws.cbufferAccum = pd.DataFrame()
    #ws.cbufferSampling = pd.DataFrame()
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

### WebSocket: Kite Integration

In [18]:
#!python
from kiteconnect import KiteTicker

#logging.basicConfig(level=logging.CRITICAL)
logger.setLevel(logging.DEBUG)


# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    ticksHandler(ws, ticks)


def on_connect(ws, response):
    initTrade(ws)
    
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(portfolioToken.tolist())

    ws.subscribe([225537])
    
    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken.tolist())
    ws.set_mode(ws.MODE_FULL, [225537]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

def on_order_update(ws, data):
    logger.info("New Order Update")
    orderNotification(ws,data)

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_order_update = on_order_update

#kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.

### WebSocket: Start/Stop

In [19]:
if mode == "Live":
    kws.connect(threaded=True)

In [21]:
if mode == "Live":
    kws.close()

In [22]:
if mode == "Live":
    print(kws.is_connected())

False


In [23]:
kite.access_token

'j97hHaYcuI0oR5Df8mJ6fJm0p3btQp8D'

### Even wait

In [76]:
import asyncio
import random
async def waiter(event):
    print('waiting for it ...')
    while(1):
        await event.wait()
        print('... got it!'+str(random.randint(1,10)))
        event.clear()
        

async def main():
    # Create an Event object.
    
    event = asyncio.Event()

    # Spawn a Task to wait until 'event' is set.
    waiter_task1 = asyncio.create_task(waiter(event))
    waiter_task2 = asyncio.create_task(waiter(event))
    waiter_task3 = asyncio.create_task(waiter(event))
    waiter_task4 = asyncio.create_task(waiter(event))

    # Sleep for 1 second and set the event.
    await asyncio.sleep(1)
    event.set()

    await asyncio.sleep(1)
    event.set()
    
    # Wait until the waiter task is finished.
    #await asyncio.gather(waiter_task1, waiter_task2, waiter_task3, waiter_task4)
    
    #event.set()

await main()

#event.set()

waiting for it ...
waiting for it ...
waiting for it ...
waiting for it ...
... got it!8
... got it!4
... got it!8
... got it!8
... got it!9
... got it!8
... got it!1
... got it!10


In [77]:
for task in asyncio.all_tasks():
    print(asyncio.gather(task))

In [ ]:
for task i​sn asyncio.all_tasks():
    task.cancel()

In [79]:
asyncio.all_tasks()

set()

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    p = Pool(5)
    print(p.map(f, [1, 2, 3]))


In [42]:
import pandas as pd
import datetime as dt



df = pd.DataFrame(data=[202,203,201,210,209,212,225], columns=['price'], 
                  index=[dt.datetime(2019,4,1),dt.datetime(2019,4,2),dt.datetime(2019,4,3),dt.datetime(2019,4,6),dt.datetime(2019,4,7),dt.datetime(2019,4,8),dt.datetime(2019,4,15)])

#df = df.resample("1D").asfreq()



In [50]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
fig = tools.make_subplots(rows=1, cols=1)

tracePrice = go.Scatter(x=df.index, y=df.price, marker = dict(color='red', size=2), 
                        mode='lines', name="Close Price", yaxis='y1')

#tickvals = [dt.datetime(2019,4,1),dt.datetime(2019,4,6),dt.datetime(2019,4,15)]
tickvals = [dt.datetime(2019,4,1),dt.datetime(2019,4,2),dt.datetime(2019,4,3),dt.datetime(2019,4,6),dt.datetime(2019,4,7),dt.datetime(2019,4,8),dt.datetime(2019,4,15)]

layout = go.Layout(
    xaxis=go.layout.XAxis(
        tickvals=tickvals,
        ticktext=tickvals
    )
)

fig['layout'] = layout

fig.append_trace(tracePrice,1,1)
#iplot(fig, filename="plot/test.html")

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [55]:
class a(object):
    def fun2():
        print("In base class")
        
class b(a):
    def fun1():
        print("In subclass")
        superfun2()
        

vara = a()
varb = b()

b.fun1()
b.fun2()
        

In subclass


NameError: name 'fun2' is not defined